# Practice on MLP
In this practical lesson, we will learn how to load a real dataset using [`pandas`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), we will normalize it with a [`MinMaxScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) and explore various approaches using [`Keras Dense layers`](https://keras.io/layers/core/#dense).

## What does the dataset contain?

In synopsis, the data are cellular variables labeled as pathogenic or not: we want to predict to which class a given datapoint is taken from.

## Loading a dataset with pandas
We will use the python package [`pandas`](http://pandas.pydata.org/pandas-docs/stable/) to load the dataset, if you need to install it just run:
```bash
pip install pandas
```
It is standard to import pandas as `pd`.

In [1]:
import pandas as pd

In [2]:
x = pd.read_csv("input.csv", index_col=0)
y = pd.read_csv("output.csv", index_col=0)

/usr/local/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


## Exploring the dataset
In a jupyter notebook is extremely simple to visualize a pandas DataFrame:

In [3]:
x

,CpGobsExp,CpGperCpG,CpGperGC,DGVCount,DnaseClusteredHyp,DnaseClusteredScore,EncH3K27Ac,EncH3K4Me1,EncH3K4Me3,GCContent,...,fantom5Robust,fracRareCommon,mamPhastCons46way,mamPhyloP46way,numTFBSConserved,priPhastCons46way,priPhyloP46way,rareVar,verPhastCons46way,verPhyloP46way
chr1.55505180,0.96,17.8,61.2,3,73,1000,39.20,15.04,33.68,0.623,...,0,0.759,0.000,-1.844,0,0.106,-1.268,22,0.001,-1.855
chr11.5246715,0.00,0.0,0.0,0,0,0,5.16,26.76,10.00,0.338,...,0,0.826,0.899,2.030,0,0.260,0.459,38,0.652,0.782
chr11.5246717,0.00,0.0,0.0,0,0,0,5.16,26.76,10.00,0.338,...,0,0.826,0.995,0.839,0,0.360,0.459,38,0.984,0.923
chr11.5246718,0.00,0.0,0.0,0,0,0,5.16,26.76,10.00,0.331,...,0,0.826,0.998,0.859,0,0.391,0.459,38,0.987,0.886
chr11.5246718,0.00,0.0,0.0,0,0,0,5.16,26.76,10.00,0.331,...,0,0.826,0.998,0.859,0,0.391,0.459,38,0.987,0.886
chr11.5246720,0.00,0.0,0.0,0,0,0,5.16,26.76,10.00,0.338,...,0,0.826,1.000,2.030,0,0.429,0.454,38,0.985,2.230
chr11.5246796,0.00,0.0,0.0,0,0,0,3.00,26.40,14.20,0.470,...,0,0.844,0.010,0.662,0,0.017,0.454,38,0.003,0.402
chr11.5248257,0.00,0.0,0.0,0,0,0,17.08,22.32,5.00,0.490,...,0,0.841,0.948,0.903,0,0.929,0.650,37,0.176,0.522
chr11.5248269,0.00,0.0,0.0,0,0,0,17.08,22.32,5.00,0.517,...,0,0.841,0.976,2.810,0,0.936,0.650,37,1.000,2.110
chr11.5248280,0.00,0.0,0.0,0,0,0,13.04,22.88,5.00,0.523,...,0,0.822,0.479,1.470,0,0.958,0.650,37,0.055,1.540


### What should I (usually) check in a dataset?

#### Nan density
In most datasets there are some unknown values, for example a patient has yet to do a blood test, so the data relative to the test in the dataset are set to `NaN`. Now, `NaN` is a special float number, which isn't equal to no other values, so if you check with an equal you will get a `False`.

In [4]:
import numpy as np

In [9]:
np.nan == np.nan

False

To check if a given value is `NaN` we have to use a function from either [`pandas`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html) or [`numpy`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.isnan.html) as follows:

In [10]:
np.isnan(np.nan), pd.isna(np.nan)

(True, True)

Measuring the `NaN` density:

In [11]:
np.mean(pd.isna(x).values)

0.0

We are lucky! Our dataset isn't sparse!

#### Data variance
Some columns could have extremely low variance, once normalized by the mean, (even 0, with constant values) and would be nearly useless for either training or testing a model:

In [14]:
variances = (x/x.mean()).var()

In [15]:
variances

CpGobsExp               187.790483
CpGperCpG               188.264205
CpGperGC                184.221247
DGVCount                  2.948082
DnaseClusteredHyp        20.708281
DnaseClusteredScore       8.568552
EncH3K27Ac               11.102579
EncH3K4Me1                2.213287
EncH3K4Me3                9.800345
GCContent                 0.056654
GerpRS                   49.061068
GerpRSpv                510.933610
ISCApath                  2.341130
commonVar                 0.414103
dbVARCount                2.948082
fantom5Perm            2486.381294
fantom5Robust          2145.723258
fracRareCommon            0.019075
mamPhastCons46way        14.844264
mamPhyloP46way            1.093232
numTFBSConserved        135.660186
priPhastCons46way         7.110275
priPhyloP46way            0.634043
rareVar                   0.086971
verPhastCons46way        12.481429
verPhyloP46way            1.190950
dtype: float64

Let's proceed to filter out the columns with less than $0.05$ variance:

In [16]:
variances[variances<0.05]

fracRareCommon    0.019075
dtype: float64

We have now identified some columns with variance so low that we could consider discarding them.

In [13]:
x = x.drop(columns=variances[variances<0.05].index)

#### Covariance and correlation
Another important tool is the correlation coefficient of the various columns: highly correlated columns add little to the training dataset:

In [18]:
correlations = x.corr()
correlations

,CpGobsExp,CpGperCpG,CpGperGC,DGVCount,DnaseClusteredHyp,DnaseClusteredScore,EncH3K27Ac,EncH3K4Me1,EncH3K4Me3,GCContent,...,fantom5Robust,fracRareCommon,mamPhastCons46way,mamPhyloP46way,numTFBSConserved,priPhastCons46way,priPhyloP46way,rareVar,verPhastCons46way,verPhyloP46way
CpGobsExp,1.000000,0.987035,0.979756,0.014144,0.373929,0.243532,0.139637,0.028156,0.382765,0.196283,...,0.004750,0.001812,0.036744,0.012239,0.035629,0.028231,0.008427,0.021246,0.035059,0.014576
CpGperCpG,0.987035,1.000000,0.990182,0.014277,0.374620,0.245349,0.138654,0.028546,0.384098,0.198141,...,0.004506,0.002152,0.034370,0.011542,0.034591,0.026364,0.007892,0.020623,0.032689,0.013674
CpGperGC,0.979756,0.990182,1.000000,0.016431,0.368252,0.242628,0.137141,0.030017,0.374522,0.197949,...,0.004312,0.001914,0.033383,0.010717,0.033638,0.025863,0.007744,0.022931,0.032730,0.013020
DGVCount,0.014144,0.014277,0.016431,1.000000,-0.005671,-0.007249,-0.009769,-0.019347,-0.002213,0.018098,...,-0.002103,-0.212403,-0.014280,-0.020333,-0.010316,-0.022123,-0.015849,0.001170,-0.003267,-0.019308
DnaseClusteredHyp,0.373929,0.374620,0.368252,-0.005671,1.000000,0.788695,0.321137,0.328771,0.405385,0.225107,...,0.058719,0.038376,0.054005,0.045076,0.046812,0.058680,0.028996,0.029058,0.040326,0.046710
DnaseClusteredScore,0.243532,0.245349,0.242628,-0.007249,0.788695,1.000000,0.288488,0.385540,0.293971,0.269701,...,0.057860,0.052116,0.058014,0.060543,0.050594,0.071325,0.038185,0.047818,0.044045,0.063083
EncH3K27Ac,0.139637,0.138654,0.137141,-0.009769,0.321137,0.288488,1.000000,0.531889,0.502775,0.131596,...,0.051893,0.029561,0.012873,0.036272,0.016798,0.020483,0.026960,0.025423,0.006862,0.036888
EncH3K4Me1,0.028156,0.028546,0.030017,-0.019347,0.328771,0.385540,0.531889,1.000000,0.217845,0.202343,...,0.047618,0.066652,0.013940,0.075608,0.022714,0.035572,0.052691,0.011948,0.005625,0.077261
EncH3K4Me3,0.382765,0.384098,0.374522,-0.002213,0.405385,0.293971,0.502775,0.217845,1.000000,0.140207,...,0.016329,0.032205,0.022408,0.034150,0.024186,0.027498,0.024898,0.023909,0.014090,0.034703
GCContent,0.196283,0.198141,0.197949,0.018098,0.225107,0.269701,0.131596,0.202343,0.140207,1.000000,...,0.011052,0.011388,-0.034160,-0.119984,-0.003642,-0.053416,-0.066847,0.177140,-0.022000,-0.120346


Some columns have high absolute correlation, so we will proceed to remove them: which one we should keep of each group? It isn't usually important, expecially when the correlation coefficient is very high.

In [23]:
x = x.drop(columns=x.columns[np.any(np.triu(correlations>0.95, k=1), axis=1)])

#### The classes are balanced?
We now proceed to check if the two classes (pathogenic and non-pathogenic) are balanced or not, meaning if the datapoints are around the same ratio:

In [27]:
np.mean(y==1)

x    0.000036
dtype: float64

The two classes are **strongly** umbalanced! We will need to use some special approaches to avoid overfitting to the class with greater cardinality.

## Splitting and normalizing a dataset
You should know how to do this by yourself! Let's see what you can do! You have 10 minutes!

In [41]:
# Task: Split the dataset
# Use this cell to split the input and output into training and testing
# Tip: remember to import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [42]:
np.sum(y_test==1), np.sum(y_train==1)

(x    7
 dtype: int64, x    34
 dtype: int64)

In [54]:
# Use this cell to normalize the data.
# Tip: remember to import MinMaxScaler from sklearn.preprocessing
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


## Creating the model
You should know how to do this by yourself! Let's see what you can do! You have 10 minutes!

In [67]:
# Task: create your own MLP
# Use this cell to create your own model
# Tip 1: remember to import Sequential, Dense and InputLayer from keras
# Tip 2: the input size can be determined by x.shape[1]
# Tip 3: remember to set the random seed!

import os
import random
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, InputLayer, Dropout
from keras.utils import print_summary

def set_seed(seed:int):
    """Set the random state of the various random extractions.
        seed:int, the seed to set the random state to.
    """
    np.random.seed(seed)
    random.seed(seed)
    tf.set_random_seed(seed)

def auroc(y_true, y_pred)->float:
    score = tf.metrics.auc(y_true, y_pred, curve="ROC", summation_method="careful_interpolation")[1]
    K.get_session().run(tf.local_variables_initializer())
    return score

def auprc(y_true, y_pred)->float:
    score = tf.metrics.auc(y_true, y_pred, curve="PR", summation_method="careful_interpolation")[1]
    K.get_session().run(tf.local_variables_initializer())
    return score
    
set_seed(42)
model = Sequential([
    InputLayer(input_shape=(x_train_scaled.shape[1],)),
    *[Dense(20, activation="relu") for i in range(3)],
    Dropout(0.5),
    *[Dense(20, activation="relu") for i in range(2)],
    Dense(1, activation="sigmoid")
])
model.compile(
    optimizer="nadam",
    loss='binary_crossentropy',
    metrics=[auroc, auprc, "accuracy"]
)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [70]:
print_summary(model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 20)                460       
_________________________________________________________________
dense_36 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_37 (Dense)             (None, 20)                420       
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_38 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_39 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 21        
Total para

## Training the model
You should know how to do this by yourself! Let's see what you can do! You have 10 minutes!

In [69]:
# Task: train your MLP
# Tip: Try using large batchsizes, not 16 or 32 but closer to 4000.
# Extra points: Can you figure our why a bigger batch size in our task can lead to better results?
model.fit(
    x_train_scaled,
    y_train,
    shuffle=True,
    verbose=1,
    validation_data=(x_test_scaled, y_test),
    epochs=100,
    batch_size=5000
)

Train on 1011932 samples, validate on 112437 samples
Epoch 1/100
1011932/1011932 [==============================] - 4s 4us/step - loss: 5.4392e-04 - auroc: 0.5006 - auprc: 3.7201e-05 - acc: 1.0000 - val_loss: 0.0010 - val_auroc: 0.5006 - val_auprc: 3.7067e-05 - val_acc: 0.9999
Epoch 2/100
1011932/1011932 [==============================] - 4s 4us/step - loss: 5.4339e-04 - auroc: 0.5006 - auprc: 3.7059e-05 - acc: 1.0000 - val_loss: 0.0010 - val_auroc: 0.5006 - val_auprc: 3.7037e-05 - val_acc: 0.9999
Epoch 3/100
1011932/1011932 [==============================] - 4s 4us/step - loss: 5.4322e-04 - auroc: 0.5006 - auprc: 3.6925e-05 - acc: 1.0000 - val_loss: 0.0010 - val_auroc: 0.5006 - val_auprc: 3.7009e-05 - val_acc: 0.9999
Epoch 4/100
1011932/1011932 [==============================] - 4s 4us/step - loss: 5.4288e-04 - auroc: 0.5005 - auprc: 3.7083e-05 - acc: 1.0000 - val_loss: 0.0010 - val_auroc: 0.5005 - val_auprc: 3.6984e-05 - val_acc: 0.9999
Epoch 5/100
1011932/1011932 [==================

Epoch 37/100
1011932/1011932 [==============================] - 4s 4us/step - loss: 3.1632e-04 - auroc: 0.5002 - auprc: 3.6714e-05 - acc: 1.0000 - val_loss: 4.7780e-04 - val_auroc: 0.5009 - val_auprc: 3.6919e-05 - val_acc: 0.9999
Epoch 38/100
1011932/1011932 [==============================] - 4s 4us/step - loss: 3.0731e-04 - auroc: 0.5017 - auprc: 3.7230e-05 - acc: 1.0000 - val_loss: 4.6811e-04 - val_auroc: 0.5022 - val_auprc: 3.7466e-05 - val_acc: 0.9999
Epoch 39/100
1011932/1011932 [==============================] - 4s 4us/step - loss: 2.9031e-04 - auroc: 0.5045 - auprc: 3.8627e-05 - acc: 1.0000 - val_loss: 4.5358e-04 - val_auroc: 0.5061 - val_auprc: 3.9440e-05 - val_acc: 0.9999
Epoch 40/100
1011932/1011932 [==============================] - 4s 4us/step - loss: 2.7349e-04 - auroc: 0.5086 - auprc: 4.1167e-05 - acc: 1.0000 - val_loss: 4.5857e-04 - val_auroc: 0.5103 - val_auprc: 4.2371e-05 - val_acc: 0.9999
Epoch 41/100
1011932/1011932 [==============================] - 4s 4us/step - lo

Epoch 73/100
1011932/1011932 [==============================] - 4s 4us/step - loss: 1.7403e-04 - auroc: 0.6410 - auprc: 9.2607e-04 - acc: 1.0000 - val_loss: 2.3696e-04 - val_auroc: 0.6425 - val_auprc: 9.5231e-04 - val_acc: 0.9999
Epoch 74/100
1011932/1011932 [==============================] - 4s 4us/step - loss: 1.7301e-04 - auroc: 0.6438 - auprc: 9.7600e-04 - acc: 1.0000 - val_loss: 2.4552e-04 - val_auroc: 0.6452 - val_auprc: 0.0010 - val_acc: 0.9999
Epoch 75/100
1011932/1011932 [==============================] - 4s 4us/step - loss: 1.5264e-04 - auroc: 0.6467 - auprc: 0.0010 - acc: 1.0000 - val_loss: 2.3548e-04 - val_auroc: 0.6480 - val_auprc: 0.0011 - val_acc: 0.9999
Epoch 76/100
1011932/1011932 [==============================] - 4s 4us/step - loss: 1.5942e-04 - auroc: 0.6493 - auprc: 0.0011 - acc: 1.0000 - val_loss: 3.0771e-04 - val_auroc: 0.6510 - val_auprc: 0.0011 - val_acc: 0.9999
Epoch 77/100
1011932/1011932 [==============================] - 4s 4us/step - loss: 1.6152e-04 - aur

## Exploring more models

### Dropout layers
With [`Dropout`](https://keras.io/layers/core/) is intented a special kind of layer, which turns off during training some neurons randomly with a given uniform probability: this is useful to reduce the risk of overfitting. Can you include this layer in your model?

In [29]:
# Task: include at least one dropout layer in your model.
# Tip: remember to import the Dropout layer from Keras

## Free experimentation: what do you want to try?
These were the basics of Keras MLPs: we will now use the remaining time to let you experiment with Keras and try to answer to every doubt about the models you encounter.